In [ ]:
import pandas

print("Importing categorized names... ")
names = pandas.read_csv("data/names_cat.csv", usecols=["name", "n_publs", "likely_gender", "score"])
print("Names imported.")

In [ ]:
# Setting index & accessing cells: https://pythonhow.com/accessing-dataframe-columns-rows-and-cells/
names = names.set_index("name", drop = False)
print(names[:10])

In [ ]:
# Get first name and last name from full name with NamSor
# Calculate the gender + score for the full name for those names with a score less than 7
# Save the new result (gender and score) in the df
# Compare the results. 
# To test if this makes sense: 
# Get sample of names with score less than 7
# send to api
# save results
# row: gender changed?: bool
# how much score changed?: int
# plot

In [ ]:
print("Getting private key... ")
# Get private API Key for NamSor API v2 (contained in txt file)
key = ''

# Import personal key
with open("key.txt", "r") as file:
    key = file.read()

if(len(key) > 0):
    print("Got private key.")
else: 
    print("Could not find private key. Please check the file name and make sure you have an API key.")

In [ ]:
# Setting up NamSor API v2 to get the gender of a name
# https://www.namsor.com/
# https://v2.namsor.com/NamSorAPIv2/apidoc.html
# using NamSor API v2 Python SDK
# https://github.com/namsor/namsor-python-sdk2
# licensed under GNU Affero General Public License v3.0

# Following script partly taken from https://github.com/namsor/namsor-python-sdk2 "Getting Started" 
# and adapted to keep key private and remove unnecessary lines.

print("Setting up NamSor API v2 connection settings...")

import openapi_client
from openapi_client.rest import ApiException

# Configure API key authorization: api_key
configuration = openapi_client.Configuration()
configuration.api_key['X-API-KEY'] = key
# create an instance of the API class
pers_api_instance = openapi_client.PersonalApi(openapi_client.ApiClient(configuration))

In [ ]:
names_to_consider = names[names['score'] <= 3].sample(1000)
print("Will be parsing and reconsidering {} names.".format(len(names_to_consider)))

In [ ]:
# clean up names
# Strip numbers from a string: https://stackoverflow.com/questions/16849109/strip-out-numbers-from-a-string
# Remove white space at end from string: https://stackoverflow.com/questions/2372573/how-do-i-remove-whitespace-from-the-end-of-a-string-in-python
import re

def c(x):
    return pandas.Series([x, re.sub(r'\d+','',x).rstrip()], index=['name', 'clean_name'])
                         
cleaned_names = names_to_consider['name'].apply(c)
print(cleaned_names)

names_to_consider = names_to_consider.set_index('name').join(cleaned_names.set_index('name'), how='left', on='name')

In [ ]:
# Formatting the names using the API's models
def createPersonalNameIn(name):
    return openapi_client.PersonalNameIn(id=name, name=name)

list_of_names = list(map(createPersonalNameIn, names_to_consider['clean_name']))

In [ ]:
print("The formatted names look like that: {}".format(list_of_names[:10]))

In [ ]:
# Now parsing the full names into first and last name, sending in
# one batch at a time and saving the result answer by answer.
# If the API calling gets interrupted:
# 1. check that no names got lost: ((len(names_to_consider)-len(result)) == len(names_stack))
# 2. If True: Restart only the code in the while loop.

batch_size = 1000 #1000 is the API limit given by NamSor
start = 0
end = batch_size
result = []

names_stack = list(names_to_consider['clean_name'])

while (len(names_stack) >= batch_size):
    try:
        current_batch = list(map(createPersonalNameIn, names_stack[start:end])) # create batch of names in correct format
        batch_personal_name_in = openapi_client.BatchPersonalNameIn(personal_names=current_batch) # convert batch to correct format
        api_response = pers_api_instance.parse_name_batch(batch_personal_name_in=batch_personal_name_in) # call API
        result = result + api_response.personal_names # save result
        
        del names_stack[start:end] # delete the names that have already been categorized from the stack
        
        # categorize remaining names if they are less than a batch size
        if(len(names_stack) < batch_size and len(names_stack) > 0):
            current_batch = list(map(createPersonalNameIn, names_stack)) # create the batch of remaining names
            batch_personal_name_in = openapi_client.BatchPersonalNameIn(personal_names=current_batch)
            api_response = pers_api_instance.parse_name_batch(batch_personal_name_in=batch_personal_name_in)
            result = result + api_response.personal_names
            names_stack = [] # empty the stack
        
        print("Batch of names analyzed")
    except ApiException as e:
        print("Exception when calling PersonalApi: gender_full_batch: %s\n" % e)


print("All batches analyzed.")

In [ ]:
# Convert results (list of openapi_client.models.PersonalNameParsedOut) to (list of dictionaries)
print('Filling the results into the names dataframe...')
for oapi_el in result:
    name = oapi_el.first_last_name
    try:
        names_to_consider.at[oapi_el.name, 'first_name'] = name.first_name
        names_to_consider.at[oapi_el.name, 'last_name'] = name.last_name
    except:
        names_to_consider.at[oapi_el.name, 'first_name'] = None
        names_to_consider.at[oapi_el.name, 'last_name'] = None
        print(name)
        print(oapi_el)
    
print('Dataframe completed with API results. Here is a sample: {}'.format(names_to_consider))

In [ ]:
names_to_consider[names_to_consider['name']=='Taehee Lee 0002']

In [ ]:
# Convert results (list of openapi_client.models.personal_name_gendered_out.PersonalNameGenderedOut) to (list of dictionaries)
print('Filling the results into the names dataframe...')
for oapi_el in result:
    names.at[oapi_el.name, 'likely_gender'] = oapi_el.likely_gender
    names.at[oapi_el.name, 'score'] = oapi_el.score
print('Dataframe completed with API results. Here is a sample: {}'.format(names[:50]))